## <center>Keras-RL DQN Model</center>


In this notebook we will create our first Reinforcement Learning agent via keras-RL2 taking the *Cartpole* as example

In [1]:
import time  # to reduce the game speed when playing manually

import gym  # Contains the game we want to play

# import necessary blocks from keras to build the Deep Learning backbone of our agent
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, Activation, InputLayer
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam  # Adam optimizer

# Now the keras-rl2 agent. Dont get confused as it is only called rl and not keras-rl
from rl.agents.dqn import DQNAgent

### a. Environment set up

In [2]:
def recall():
    env = gym.make('CartPole-v1')
    return env

env = recall()
env.reset()

for _ in range(9):
    env.render(mode="human")  
    random_action = env.action_space.sample()
    env.step(random_action)

env.close()

In [3]:
num_actions = env.action_space.n
num_observations = env.observation_space.shape[0]
print(f"There are {num_actions} possible actions and {num_observations} observations")

There are 2 possible actions and 4 observations


### b. DQN agent set up
The DQN agent created with keras-RL2, needs the following parameters to be created:

**1. Model**

The model is the ANN, in this case we will use the same as the one implemented in the Manual DQN notebook

In [4]:
model = Sequential()
model.add(InputLayer(input_shape=(1, num_observations)))
model.add(Flatten())

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(num_actions))
model.add(Activation('linear'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 16)                80        
                                                                 
 activation (Activation)     (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 activation_1 (Activation)   (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
 activation_2 (Activation)   (None, 2)                 0

**2. nb_actions**

Number of actions --> already defined

**3. memory**

The action replay memory. You can choose between the *SequentialMemory()* and *EpisodeParameterMemory()* which is only used for one RL agent called *CEM*. Sequential Memory is for storing observations (optimized circular buffer)

Here we initialize the circular buffer with a limit of 20000 and a window length of 1. The window length describes the number of subsequent actions stored for a state. This will be demonstrated in the next lecture, when we start dealing with images


In [5]:
from rl.memory import SequentialMemory  
memory = SequentialMemory(limit=20000, window_length=1)

**4. nb_steps_warmup**

How many iterations without training - Used to fill the memory

**5. target_model_update**

When do we update the target model?

**6. Action Selection policy**

There are many policies to chose from, some of them like the *LinearAnnealedPolicy()*, are referred as outter policies and take an inner policy such as *SoftmaxPolicy()*, *EpsGreedyQPolicy()*, *GreedyQPolicy()*, *GreedyQPolicy()*, *MaxBoltzmannQPolicy()* and *BoltzmannGumbelQPolicy()*. 

In [6]:
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), # inner policy
                              attr='eps', # attribute 
                              value_max=1.0, # max value of the attribute
                              value_min=0.001, # min value of the attribute 
                              value_test=0.0005, # small value to test the model --> explotation
                              nb_steps=200000) 

Now we create the DQN Agent based on the defined model (**model**), the possible actions (**nb_actions**) (left and right in this case), the circular buffer (**memory**), the burnin or warmup phase (**10**), how often the target model gets updated (**100**) and the policy (**policy**)


In [7]:
dqn = DQNAgent(model=model, 
               nb_actions=num_actions, 
               memory=memory, 
               nb_steps_warmup=10,
               target_model_update=100, 
               policy=policy)

# Compilation
dqn.compile(Adam(learning_rate=0.0001), metrics=['mae']) 

# Now we run the training for 20000 steps. You can change visualize=True if you want to watch your model learning. 
# Keep in mind that this increases the running time

dqn.fit(env, nb_steps=200000, visualize=False, verbose=0)

c:\Users\Lenovo\anaconda3\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\Users\Lenovo\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


In [8]:
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 232.000, steps: 232
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 283.000, steps: 283
Episode 4: reward: 334.000, steps: 334
Episode 5: reward: 232.000, steps: 232
